# Exploring the Current Inventory

In this notebook, I will explore several relevant tables to identify and count the current products in each of the company's warehouses.

## Warehouse Information

I'll start by exploring the number of warehouses. Based on the results of the query below, we can see that there are 4 warehouses distributed in the North, South, East, and West areas.

In [9]:
SELECT * FROM mintclassics.warehouses

(4 row(s) affected)

Total execution time: 00:00:01.053

warehouseCode,warehouseName,warehousePctCap
a,North,72
b,East,67
c,West,50
d,South,75


## Product Information

The `product` table contains relevant details about each unique product. Based on the results of the below query, we have 110 products and a total of 555131 stocks across all warehouses.

Upon checking the distribution of stocks per warehouse, it was found that the number of stocks in each warehouse was not evenly distributed. For instance, 39.48% of stocks were stored in the East warehouse. The north and west warehouse has an even amount of stocks.  The south warehouse contains the least amount of stocks comprising only 14.30%.

With this imbalanced stock allocation, is closing the south warehouse a practical choice? How should we redistribute the stocks across the remaining three warehouses? Let's try to address these questions next.

In [14]:
SELECT * FROM mintclassics.products
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:01.010

productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP
S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front suspension, gear-shift lever, footbrake lever, drive chain, wheels and steering. All parts are particularly delicate due to their precise scale and require special care and attention.",7933,a,48.81,95.70
S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,7305,b,98.58,214.30
S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish.",6625,a,68.99,118.94
S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos and insignias, detachable rear wheelie bar, heavy diecast metal with resin parts, authentic multi-color tampo-printed graphics, separate engine drive belts, free-turning front fork, rotating tires and rear racing slick, certificate of authenticity, detailed engine, display stand , precision diecast replica, baked enamel finish, 1:10 scale model, removable fender, seat and tank cover piece for displaying the superior detail of the v-twin engine",5582,a,91.02,193.66
S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,3252,b,85.68,136.00
S10_4962,1962 LanciaA Delta 16V,Classic Cars,1:10,Second Gear Diecast,Features include: Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,6791,b,103.42,147.74
S12_1099,1968 Ford Mustang,Classic Cars,1:12,Autoart Studio Design,"Hood, doors and trunk all open to reveal highly detailed interior features. Steering wheel actually turns the front wheels. Color dark green.",68,b,95.34,194.57
S12_1108,2001 Ferrari Enzo,Classic Cars,1:12,Second Gear Diecast,Turnable front wheels; steering function; detailed interior; detailed engine; opening hood; opening trunk; opening doors; and detailed chassis.,3619,b,95.59,207.80
S12_1666,1958 Setra Bus,Trucks and Buses,1:12,Welly Diecast Productions,"Model features 30 windows, skylights & glare resistant glass, working steering system, original logos",1579,d,77.90,136.67
S12_2823,2002 Suzuki XREO,Motorcycles,1:12,Unimax Art Galleries,"Official logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish.",9997,a,66.27,150.62


In [15]:
-- Counting the total number of products across all warehouses --
SELECT
    COUNT(productCode) AS numberOfProducts,
    SUM(quantityInStock) AS quantityInStock
FROM mintclassics.products

(1 row(s) affected)

Total execution time: 00:00:01.015

numberOfProducts,quantityInStock
110,555131


In [19]:
-- Categorizing the number of stocks by `warehouseCode`--

SELECT
    b.warehouseName,
    COUNT(productCode) as numberOfProducts,
    SUM(quantityInStock) as quantityInStock,
    SUM(quantityInStock) / (SELECT SUM(quantityInStock) FROM mintclassics.products) as proportionOfTotalStock
FROM mintclassics.products as a
JOIN mintclassics.warehouses as b ON a.warehouseCode = b.warehouseCode 
GROUP BY b.warehouseName
ORDER BY proportionOfTotalStock DESC

(4 row(s) affected)

Total execution time: 00:00:01.028

warehouseName,numberOfProducts,quantityInStock,proportionOfTotalStock
East,38,219183,0.3948
North,25,131688,0.2372
West,24,124880,0.2250
South,23,79380,0.1430


In [35]:
-- Categorizing the number of stocks by `warehouseCode` and `productLine` --

SELECT
    b.warehouseName,
    productLine,
    COUNT(productCode) as numberOfProducts,
    SUM(quantityInStock) as quantityInStock,
    SUM(quantityInStock) / (SELECT SUM(quantityInStock) FROM mintclassics.products) as proportionOfTotalStock
FROM mintclassics.products as a
JOIN mintclassics.warehouses as b ON a.warehouseCode = b.warehouseCode
GROUP BY b.warehouseName, productLine
ORDER BY proportionOfTotalStock DESC

(7 row(s) affected)

Total execution time: 00:00:01.011

warehouseName,productLine,numberOfProducts,quantityInStock,proportionOfTotalStock
East,Classic Cars,38,219183,0.3948
West,Vintage Cars,24,124880,0.2250
North,Motorcycles,13,69401,0.1250
North,Planes,12,62287,0.1122
South,Trucks and Buses,11,35851,0.0646
South,Ships,9,26833,0.0483
South,Trains,3,16696,0.0301


## Considering the Warehouse Capacity and the Amount of Stocks

Before we decide to close the south warehouse, let's try to have a deeper understanding of the stock capacity of each warehouse. The result of the query below shows that the reason why the south warehouse contains the least amount of stocks is that it has the smallest capacity out of the four.

In [33]:
SELECT
    b.warehouseName,
    ANY_VALUE((b.warehousePctCap / 100)) as propCapConsumed,
    SUM(quantityInStock) as quantityInStock,
    ANY_VALUE(CAST(ROUND(SUM(quantityInStock) / (b.warehousePctCap / 100)) AS UNSIGNED)) as spaceRemaining,
    ANY_VALUE(SUM(quantityInStock) + CAST(ROUND(SUM(quantityInStock) / (b.warehousePctCap / 100)) AS UNSIGNED)) as totalCapacity
FROM mintclassics.products as a
JOIN mintclassics.warehouses as b ON a.warehouseCode = b.warehouseCode 
GROUP BY b.warehouseName
ORDER BY totalCapacity DESC

(4 row(s) affected)

Total execution time: 00:00:01.006

warehouseName,propCapConsumed,quantityInStock,spaceRemaining,totalCapacity
East,0.67,219183,327139,546322
West,0.5,124880,249760,374640
North,0.72,131688,182900,314588
South,0.75,79380,105840,185220


## Which Warehouse to Use

Now let's see where we could best fit the stocks at the south warehouse using only the remaining space of the other warehouses. Based on the results of the below query, it can be found that the east warehouse still contains plenty of space even if we move all of the stocks from the south warehouse.

In [39]:
SELECT
    b.warehouseName,
    ANY_VALUE(CAST(ROUND(SUM(quantityInStock) / (b.warehousePctCap / 100)) AS UNSIGNED)) as spaceRemaining,
    ANY_VALUE((ROUND(SUM(quantityInStock) / (b.warehousePctCap / 100)) - 185220)) as remainingSpaceAferMovement
FROM mintclassics.products as a
JOIN mintclassics.warehouses as b ON a.warehouseCode = b.warehouseCode
WHERE b.warehouseName <> 'South'
GROUP BY b.warehouseName
ORDER BY remainingSpaceAferMovement DESC

(3 row(s) affected)

Total execution time: 00:00:01.010

warehouseName,spaceRemaining,remainingSpaceAferMovement
East,327139,141919.0
West,249760,64540.0
North,182900,-2320.0


## Conclusions

In this notebook, we explored the distribution of products in each warehouse. By using the warehouse capacity as the basis of analysis, we were able to find out that the south warehouse does not hold much of the stocks and thus, its contents could be moved to the east warehouse where there was still so much room available.

Before we finally conclude to close and move the contents of the south warehouse to the east warehouse, let's first consider other factors that may influence inventory reorganization and reduction. This will be the topic of the next notebook.